## Use tensorflow 1.4.0

In [1]:
#!pip install tensorflow_probability

In [2]:
import numpy as np
import tensorflow as tf
#import tensorflow.keras as tfk
#from tensorflow import keras as tfk
from astroNN.models import ApogeeCNN, ApogeeBCNN
from astroNN.datasets import H5Loader
from astroNN.nn.losses import mean_squared_error, mean_absolute_error, mean_error
import matplotlib.pyplot as plt

/anaconda3/envs/tf.1.4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/envs/tf.1.4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/envs/tf.1.4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/envs/tf.1.4/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [3]:
# import everything we need
from astroNN.models import load_folder
# load the pretrained base CNN and BCNN model
tf.reset_default_graph()

Bnet = load_folder("astroNN_spectra_paper_figures/astroNN_0606_run001")

Loaded astroNN model, model type: Bayesian Convolutional Neural Network -> ApogeeBCNN


In [4]:
Bnet.labels_norm_mode

2

In [5]:
Bnet.keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 7514, 1)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 7514, 2)      18          input[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 7514, 2)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
MCDropout_1 (MCDropout)         (None, 7514, 2)      0           activation_1[0][0]               
__________________________________________________________________________________________________
conv1d_2 (

In [6]:
# create a BCNN instance and then assign pretrained weights
import numpy as np
import tensorflow as tf
from astroNN.config import keras_import_manager
tfk = keras_import_manager()
from astroNN.apogee import aspcap_mask
from astroNN.apogee.plotting import ASPCAP_plots
from astroNN.models.base_bayesian_cnn import BayesianCNNBase
from astroNN.models.base_cnn import CNNBase
from astroNN.models.base_vae import ConvVAEBase
from astroNN.nn.layers import MCDropout, BoolMask, StopGrad, KLDivergenceLayer
from astroNN.nn.losses import bayesian_binary_crossentropy_wrapper, bayesian_binary_crossentropy_var_wrapper
from astroNN.nn.losses import bayesian_categorical_crossentropy_wrapper, bayesian_categorical_crossentropy_var_wrapper
from astroNN.nn.losses import mse_lin_wrapper, mse_var_wrapper

Add = tfk.layers.Add
Dense = tfk.layers.Dense
Input = tfk.layers.Input
Conv1D = tfk.layers.Conv1D
Lambda = tfk.layers.Lambda
Reshape = tfk.layers.Reshape
Dropout = tfk.layers.Dropout
Flatten = tfk.layers.Flatten
Multiply = tfk.layers.Multiply
Activation = tfk.layers.Activation
concatenate = tfk.layers.concatenate
MaxPooling1D = tfk.layers.MaxPooling1D

Model = tfk.models.Model
Sequential = tfk.models.Sequential

regularizers = tfk.regularizers
MaxNorm = tfk.constraints.MaxNorm
RandomNormal = tfk.initializers.RandomNormal


class ApogeeBCNN(BayesianCNNBase, ASPCAP_plots):
    """
    Class for Bayesian convolutional neural network for stellar spectra analysis

    :History: 2017-Dec-21 - Written - Henry Leung (University of Toronto)
    """

    def __init__(self, lr=0.0005, dropout_rate=0.3):
        super().__init__()

        self._implementation_version = '1.0'
        self.initializer = RandomNormal(mean=0.0, stddev=0.05)
        self.activation = 'relu'
        self.num_filters = [2, 4]
        self.filter_len = 8
        self.pool_length = 4
        self.num_hidden = [256, 128] #based on paper
        self.max_epochs = 100
        self.lr = 0.0005
        self.reduce_lr_epsilon = 0.00005

        self.reduce_lr_min = 1e-8
        self.reduce_lr_patience = 2
        self.l2 = 5e-9
        self.dropout_rate = 0.3

        self.input_norm_mode = 3 #center features (labels only)

        self.task = 'regression'

        self.targetname = ['teff', 'logg', 'C', 'C1', 'N', 'O', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'K',
                  'Ca', 'Ti', 'Ti2', 'V', 'Cr', 'Mn', 'Fe','Co', 'Ni']
    def model(self):
        input_tensor = Input(shape=self._input_shape, name='input')
        labels_err_tensor = Input(shape=(self._labels_shape,), name='labels_err')

        cnn_layer_1 = Conv1D(kernel_initializer=self.initializer, padding="same", filters=self.num_filters[0],
                             kernel_size=self.filter_len, kernel_regularizer=regularizers.l2(self.l2))(input_tensor)
        activation_1 = Activation(activation=self.activation)(cnn_layer_1)
        dropout_1 = MCDropout(self.dropout_rate, disable=self.disable_dropout)(activation_1)
        cnn_layer_2 = Conv1D(kernel_initializer=self.initializer, padding="same", filters=self.num_filters[1],
                             kernel_size=self.filter_len, kernel_regularizer=regularizers.l2(self.l2))(dropout_1)
        activation_2 = Activation(activation=self.activation)(cnn_layer_2)
        maxpool_1 = MaxPooling1D(pool_size=self.pool_length)(activation_2)
        flattener = Flatten()(maxpool_1)
        dropout_2 = MCDropout(self.dropout_rate, disable=self.disable_dropout)(flattener)
        layer_3 = Dense(units=self.num_hidden[0], kernel_regularizer=regularizers.l2(self.l2),
                        kernel_initializer=self.initializer)(dropout_2)
        activation_3 = Activation(activation=self.activation)(layer_3)
        dropout_3 = MCDropout(self.dropout_rate, disable=self.disable_dropout)(activation_3)
        layer_4 = Dense(units=self.num_hidden[1], kernel_regularizer=regularizers.l2(self.l2),
                        kernel_initializer=self.initializer)(dropout_3)
        activation_4 = Activation(activation=self.activation)(layer_4)
        output = Dense(units=self._labels_shape, activation=self._last_layer_activation, name='output')(activation_4)
        variance_output = Dense(units=self._labels_shape, activation='linear', name='variance_output')(activation_4)

        model = Model(inputs=[input_tensor, labels_err_tensor], outputs=[output, variance_output])
        # new astroNN high performance dropout variational inference on GPU expects single output
        model_prediction = Model(inputs=[input_tensor], outputs=concatenate([output, variance_output]))

        if self.task == 'regression':
            variance_loss = mse_var_wrapper(output, labels_err_tensor)
            output_loss = mse_lin_wrapper(variance_output, labels_err_tensor)
        elif self.task == 'classification':
            output_loss = bayesian_categorical_crossentropy_wrapper(variance_output)
            variance_loss = bayesian_categorical_crossentropy_var_wrapper(output)
        elif self.task == 'binary_classification':
            output_loss = bayesian_binary_crossentropy_wrapper(variance_output)
            variance_loss = bayesian_binary_crossentropy_var_wrapper(output)
        else:
            raise RuntimeError('Only "regression", "classification" and "binary_classification" are supported')

        return model, model_prediction, output_loss, variance_loss


In [7]:
import numpy as np

from astroNN.datasets import H5Loader
from astroNN.apogee import aspcap_mask
from astroNN.models import ApogeeBCNNCensored

# Load the training data
loader = H5Loader('/Users/nhuang37/Desktop/NYU DS/Yr 2 Sem1/Capstone/AstroNN/astroNN_spectra_paper_figures/__train_r_giant_1k_8575_1125_1557.h5')
loader.load_err = True
loader.target = ['teff', 'logg', 'C', 'C1', 'N', 'O', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'K',
                  'Ca', 'Ti', 'Ti2', 'V', 'Cr', 'Mn', 'Fe','Co', 'Ni']
x,y,x_err,y_err = loader.load()


/anaconda3/envs/tf.1.4/lib/python3.6/site-packages/astroNN/datasets/h5.py:581: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(self.h5path) as F:  # ensure the file will be cleaned up
/anaconda3/envs/tf.1.4/lib/python3.6/site-packages/astroNN/datasets/h5.py:613: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(self.h5path) as F:  # ensure the file will be cle

In [8]:
from astroNN.apogee import visit_spectra, combined_spectra, gap_delete, apogee_continuum, bitmask_boolean
x = gap_delete(x, dr=14)
x_err = gap_delete(x_err, dr=14)
x.shape, x_err.shape

((1000, 7514), (1000, 7514))

In [9]:
##create the same model instance as pretrained in tf2.0 and thenload weight in tf14
final_model = ApogeeBCNN()
final_model.max_epochs = 1
#compile the model by calling train (error after 1 epoch)
final_model.train(x,y,labels_err=y_err)

Number of Training Data: 900, Number of Validation Data: 100
====Message from Normalizer====
You selected mode: 3
Featurewise Center: True
Datawise Center: False
Featurewise std Center: False
Datawise std Center: False
====Message ends====
====Message from Normalizer====
You selected mode: 2
Featurewise Center: True
Datawise Center: False
Featurewise std Center: True
Datawise std Center: False
====Message ends====
Epoch 1/1
 - 3s - loss: 0.5527 - output_loss: 0.5527 - variance_output_loss: 0.5527 - output_mean_absolute_error: 0.8281 - output_mean_error: -1.0936e-02 - val_loss: 0.3714 - val_output_loss: 0.3714 - val_variance_output_loss: 0.3714 - val_output_mean_absolute_error: 0.6815 - val_output_mean_error: 0.0724
Completed Training, 3.50s in total


In [10]:
for layer in final_model.keras_model.layers:
    g=layer.get_config()
    weights = layer.get_weights()
    print (g)
    print(weights)

{'batch_input_shape': (None, 7514, 1), 'dtype': 'float32', 'sparse': False, 'name': 'input'}
[]
{'name': 'conv1d_3', 'trainable': True, 'filters': 2, 'kernel_size': (8,), 'strides': (1,), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1,), 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'RandomNormal', 'config': {'mean': 0.0, 'stddev': 0.05, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': {'class_name': 'L1L2', 'config': {'l1': 0.0, 'l2': 4.999999969612645e-09}}, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
[array([[[ 0.04328403, -0.02489979]],

       [[-0.04168755,  0.05353523]],

       [[-0.02089618,  0.02095718]],

       [[-0.01685653, -0.0070611 ]],

       [[-0.02529809,  0.0210314 ]],

       [[-0.04014603,  0.09160174]],

       [[-0.00785434,  0.04564562]],

       [[-0.00188191,  0.03510518]]], dtype=float32), a

In [11]:
#load the pretrained accurate weights into final_model.model
final_model.keras_model.load_weights('astroNN_spectra_paper_figures/astroNN_0606_run001/model_weights.h5')

In [12]:
for layer in final_model.keras_model.layers:
    g=layer.get_config()
    weights = layer.get_weights()
    print (g)
    print(weights)

{'batch_input_shape': (None, 7514, 1), 'dtype': 'float32', 'sparse': False, 'name': 'input'}
[]
{'name': 'conv1d_3', 'trainable': True, 'filters': 2, 'kernel_size': (8,), 'strides': (1,), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1,), 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'RandomNormal', 'config': {'mean': 0.0, 'stddev': 0.05, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': {'class_name': 'L1L2', 'config': {'l1': 0.0, 'l2': 4.999999969612645e-09}}, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
[array([[[-0.09652549, -0.2489283 ]],

       [[ 0.01814175, -0.18375222]],

       [[ 0.14887606, -0.18402125]],

       [[ 0.1334438 , -0.27137133]],

       [[ 0.23086593, -0.1801472 ]],

       [[ 0.18287252, -0.01084171]],

       [[ 0.2220627 ,  0.01960662]],

       [[ 0.2190144 ,  0.33213758]]], dtype=float32), a

{'name': 'dense_4', 'trainable': True, 'units': 128, 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'RandomNormal', 'config': {'mean': 0.0, 'stddev': 0.05, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': {'class_name': 'L1L2', 'config': {'l1': 0.0, 'l2': 4.999999969612645e-09}}, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
[array([[-0.02101721,  0.03994253,  0.02460754, ..., -0.00849252,
        -0.20806563,  0.01849252],
       [-0.00849423, -0.02787474, -0.03498743, ...,  0.06283721,
         0.00715464,  0.00842208],
       [ 0.16062595, -0.05438112,  0.03290578, ...,  0.02805254,
        -0.00506102,  0.02321132],
       ...,
       [ 0.03317602, -0.2532982 ,  0.06711821, ...,  0.03575029,
        -0.00371278, -0.05488677],
       [-0.06703904, -0.02719532, -0.10338812, ..., -0.0230916 ,
        -0.33997416, -0.02887852],
       [ 0.0724702

In [13]:
print(final_model.keras_model.inputs)
print(final_model.keras_model.outputs)

[<tf.Tensor 'input_1:0' shape=(?, 7514, 1) dtype=float32>, <tf.Tensor 'labels_err_1:0' shape=(?, 22) dtype=float32>]
[<tf.Tensor 'output_1/BiasAdd:0' shape=(?, 22) dtype=float32>, <tf.Tensor 'variance_output_1/BiasAdd:0' shape=(?, 22) dtype=float32>]


In [14]:
#output pretrained BCNN model to TF file
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

In [15]:
import keras
keras.__version__

'2.2.4'

In [16]:
# Write the graph in binary .pb file
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

outdir = "model"
prefix = "bcnn" 
name = 'output_graph.pb'
# Alias the outputs in the model - this sometimes makes them easier to access in TF
pred = []
pred_node_names = []
for i, o in enumerate(final_model.keras_model.outputs):
        pred_node_names.append(prefix+'_'+str(i))
        pred.append(tf.identity(o, 
                                name=pred_node_names[i]))
print('Output nodes names are: ', pred_node_names)


sess = K.get_session()
    
# Write the graph in human readable
# f = 'graph_def_for_reference.pb.ascii'
# tf.train.write_graph(sess.graph.as_graph_def(), outdir, f, as_text=True)
# print('Saved the graph definition in ascii format at: ',  os.path.join(outdir, f))

constant_graph = graph_util.convert_variables_to_constants(sess, 
                                                           sess.graph.as_graph_def(), 
                                                           pred_node_names)
graph_io.write_graph(constant_graph, outdir, name, as_text=False)
## Finally delete the Keras's session
K.clear_session()

Output nodes names are:  ['bcnn_0', 'bcnn_1']
INFO:tensorflow:Froze 12 variables.
Converted 12 variables to const ops.


## Made Attacks on BCNN

In [20]:
#load model back 
import os, sys 
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

outdir = "model"
#name = 'BCNN_astroNN_0606_run001.pb'
name = 'output_graph.pb'

## Make sure that there are no defult graph

tf.reset_default_graph()

def load_graph(model_name):
    #graph = tf.Graph()
    graph = tf.get_default_graph()
    graph_def = tf.GraphDef()
    with open(model_name, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)
    return graph

my_graph = load_graph(model_name=os.path.join(outdir, name))

In [21]:
for i, op in enumerate(tf.get_default_graph().get_operations()):
    print ("{: 3.0f}: {}".format(i,op.name) )

  0: import/input_1
  1: import/conv1d_3/kernel
  2: import/conv1d_3/kernel/read
  3: import/conv1d_3/bias
  4: import/conv1d_3/bias/read
  5: import/conv1d_3/convolution/ExpandDims/dim
  6: import/conv1d_3/convolution/ExpandDims
  7: import/conv1d_3/convolution/ExpandDims_1/dim
  8: import/conv1d_3/convolution/ExpandDims_1
  9: import/conv1d_3/convolution/Conv2D
 10: import/conv1d_3/convolution/Squeeze
 11: import/conv1d_3/Reshape/shape
 12: import/conv1d_3/Reshape
 13: import/conv1d_3/add
 14: import/activation_5/Relu
 15: import/MCDropout_4/dropout/keep_prob
 16: import/MCDropout_4/dropout/Shape
 17: import/MCDropout_4/dropout/random_uniform/min
 18: import/MCDropout_4/dropout/random_uniform/max
 19: import/MCDropout_4/dropout/random_uniform/RandomUniform
 20: import/MCDropout_4/dropout/random_uniform/sub
 21: import/MCDropout_4/dropout/random_uniform/mul
 22: import/MCDropout_4/dropout/random_uniform
 23: import/MCDropout_4/dropout/add
 24: import/MCDropout_4/dropout/Floor
 25: imp

In [22]:
# %load keras-tf-pb/show_graph.py
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import tensorflow as tf
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, encoding='utf-8')
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [23]:
#from show_graph import show_graph
from keras import backend as K
import tensorflow as tf
sess = K.get_session()
graph_def = sess.graph.as_graph_def()
# graph_def
show_graph(graph_def)

In [24]:
#can also attack variance: 
input_op = my_graph.get_operation_by_name("import/input_1") #the only input node available in the crazy graph
output_op = my_graph.get_operation_by_name("import/bcnn_0") #get the classification logits
ops = (input_op,output_op)

In [25]:
#check input/output operation node shape
input_op.outputs[0], output_op.outputs[0]

(<tf.Tensor 'import/input_1:0' shape=(?, 7514, 1) dtype=float32>,
 <tf.Tensor 'import/bcnn_0:0' shape=(?, 22) dtype=float32>)

In [26]:
#source_PGD: https://github.com/MadryLab/mnist_challenge/blob/master/pgd_attack.py
#source: https://fairyonice.github.io/Generate-adversarial-examples-using-TensorFlow.html
from keras import backend as K
import numpy as np

class AdversarialFlux(object): 
    
    def __init__(self,inp,out,eps=0.01, method='FGSM', iteration=1, restart=None, lr='eps'):
        '''
        inp : input tensor  (tr_flux
        out : output tensor (y_pred)
        eps : scalar
        '''
        self.inp = inp.outputs[0]
        self.out = out.outputs[0]
        self.method = method
        self.lr = lr #for PGD step size per iteration (#pervious batch, set lr=1/4 eps)
        self.define_aimage_tensor(float(eps))
        self.k = iteration
        self.eps = eps #for FGSM step size; PGD radius
        self.restart = restart
        
    def mse_tf(self,y_true,y_pred,verbose=True):
        '''
        y_pred : tensor 
        y_test : tensor having the same shape as y_pred
        ##should pass in normalized (centered y)
        '''
        ## preserve the same shape as y_pred.shape
        square = tf.square(y_true - y_pred) 
        ## mean across the final dimensions
        ms = tf.reduce_mean(square)
        return(ms)

    def define_aimage_tensor(self,eps):
        '''
        Define a graph to output adversarial image

        Xnew = X + eps * sign(dX)
        '''
        ### deterministic
        K.set_learning_phase(0)

        ## get list of target
        yshape = [None] + [int(i) for i in self.out.get_shape()[1:]]
        y_true_tf = tf.placeholder(tf.float32, yshape)
        y_pred_tf = self.out 
        
        #loss = self.mse_tf(y_pred_tf,y_true_tf) 
        #only on the subset 3 labels
        y_pred_sub =  tf.stack([y_pred_tf[:,0], y_pred_tf[:,1],y_pred_tf[:,19]], axis=1)
        y_true_sub = tf.stack([y_true_tf[:,0], y_true_tf[:,1],y_true_tf[:,19]], axis=1)
        loss = self.mse_tf(y_true_sub,y_pred_sub) 

        ## tensor that calculate the gradient of loss with respect to image i.e., dX
        grad_tf          = tf.gradients(loss,self.inp)[0]
        grad_sign_tf     = tf.sign(grad_tf)
        
        if self.method == 'FGSM':
            eps_tf = tf.constant(eps,name="epsilon")
            grad_sign_eps_tf = tf.scalar_mul(eps_tf,grad_sign_tf)
        else: #PGD
            if self.lr == 'eps':
                eps_tf = tf.constant(eps/4,name="epsilon") #step size = 1/4 radius, previous batch of results
            else:
                eps_tf = tf.constant(self.lr,name="epsilon")
            grad_sign_eps_tf = tf.scalar_mul(eps_tf,grad_sign_tf)
        
        new_image_tf = tf.add(grad_sign_eps_tf,self.inp)
  
        self.y_true  = y_true_tf
        self.eps     = eps_tf
        self.aimage  = new_image_tf
        self.added_noise = grad_sign_eps_tf
        self.loss = loss #return loss

               
    
    def predict(self,X):

        with tf.Session() as sess:
            y_pred = sess.run(self.out,
                              feed_dict={self.inp:X})
        return(y_pred)
    
    def get_aimage(self,X_nat,y,added_noise=False,loss=False): #FGSM versus PGD
        tensor2eval = [self.aimage]
        if added_noise:
            tensor2eval.append(self.added_noise)
        if loss:
            tensor2eval.append(self.loss)
        #print(X_nat.shape)
        X = np.copy(X_nat)
        with tf.Session() as sess:
            if self.method == 'FGSM':
                X, noise, loss = sess.run(tensor2eval,feed_dict={self.inp:X,self.y_true:y})
                X = X.reshape(*X_nat.shape)
                return([X, noise, loss])
            elif self.method=='PGD' and self.restart is None: #PGD
                for k in range(self.k):#k-1 more iterations
                    X, noise = sess.run(tensor2eval,feed_dict={self.inp:X,self.y_true:y})
                    X = np.clip(X,X_nat - self.eps, X_nat + self.eps )
                    #print(X.shape)
                X = X.reshape(*X_nat.shape)
                noise = noise.reshape(*X_nat.shape)
                return([X,noise])
            else: #PGD with random restart
                best_RMSE = 0
                best_X_perturb = None
                best_noise = None
                for r in range(self.restart): #number of restarts
                    print('restart'+str(r))
                    X = X_nat + np.random.uniform(-self.eps, self.eps, X_nat.shape) #restart with random perturbation within L_infinity ball
                    for k in range(self.k):#k-1 more iterations
                        X, noise, RMSE = sess.run(tensor2eval,feed_dict={self.inp:X,self.y_true:y})
                        X = np.clip(X,X_nat - self.eps, X_nat + self.eps )
                        #evaluate robust MSE loss (simple, on rescale set, no MCMC inference)
                        print(np.sqrt(RMSE))
                        if RMSE > best_RMSE:
                            best_RMSE = RMSE
                            best_X_perturb = X
                            best_noise = noise
                    print("this restart has best RMSE as ", best_RMSE)
                return([best_X_perturb.reshape(*X_nat.shape),best_noise.reshape(*X_nat.shape),best_RMSE])


In [27]:
x.shape

(1000, 7514)

### FGSM attack

In [29]:
size = np.concatenate((np.around(np.linspace(0.001, 0.01, num=10),decimals=3), 
                      np.around(np.linspace(0.02, 0.1, num=9),decimals=2)))
size = size[:11]
size

array([0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009,
       0.01 , 0.02 ])

In [32]:
#attack target: y_predi (instead of original y)
#load previously saved predictions
import pickle
with open('rerun_1k_new/adv_preds_random.pkl', 'rb') as f:
    adv_random = pickle.load(f)
#get result of normal pred
y_predi, y_predi_std = adv_random['y_predi'],adv_random['y_predi_std']

In [33]:
y_predi.shape

(1000, 22)

In [37]:
#iterate different attack size and output, and savedict to dump pickle
results_scaled = {}
y_p_mean = np.mean(y_predi,axis=0)
y_scaled= y_predi - y_p_mean
for eps in size:
    AIbadder  = AdversarialFlux(*ops,eps=eps)
    Xi =  x.reshape(x.shape[0], x.shape[1], 1)
    X_bad, noise,loss   = AIbadder.get_aimage(Xi,y_scaled,added_noise=True,loss=True)
    print(np.sqrt(loss))
    X_bad = X_bad.reshape(x.shape)
    results_scaled[eps] = X_bad

201.36476
201.38283
201.44275
201.34221
201.39441
201.36368
201.36134
201.36945
201.37312
201.3327
201.3736


In [38]:
#output rmse_bad and uncertainty_bad to plot against random
with open('rerun_1k_MSE/attack_dict_fgsm_scaled_tar.pkl', 'wb') as f:
    pickle.dump(results_scaled, f)

### PGD attacks w/ random restarts

In [87]:
#pgd: it = 10, 
results_pgd = {}
for eps in size: 
    AIbadder  = AdversarialFlux(*ops,eps=eps, method='PGD', iteration=10, restart=5, lr='eps')
    Xi =  x.reshape(x.shape[0], x.shape[1], 1)
    (X_bad, noise_bad,loss)   = AIbadder.get_aimage(Xi,y_scaled,added_noise=True,loss=True) #(Xi,y,added_noise=True)
    X_bad = X_bad.reshape(x.shape)
    results_pgd[eps] = X_bad

restart0
168.44373
168.48572
168.41283
168.38187
168.43558
168.42833
168.45355
168.50043
168.30695
168.40327
this restart has best RMSE as  28392.395
restart1
168.44696
168.39705
168.46863
168.36642
168.42125
168.40184
168.43208
168.41206
168.38666
168.39674
this restart has best RMSE as  28392.395
restart2
168.38516
168.4239
168.44206
168.42873
168.42358
168.42787
168.41898
168.42523
168.4568
168.41833
this restart has best RMSE as  28392.395
restart3
168.42574
168.44016
168.39767
168.42305
168.45439
168.4415
168.4369
168.40883
168.41327
168.4146
this restart has best RMSE as  28392.395
restart4
168.3922
168.39963
168.38954
168.41858
168.41765
168.40366
168.40001
168.38994
168.43121
168.45856
this restart has best RMSE as  28392.395
restart0
168.46098
168.40738
168.44066
168.39041
168.4182
168.42429
168.45828
168.44272
168.42378
168.43533
this restart has best RMSE as  28379.104
restart1
168.42729
168.41875
168.45334
168.35521
168.46356
168.41878
168.4977
168.42668
168.52452
168.45992

168.78433
168.77203
168.85522
168.92781
168.91458
168.93799
this restart has best RMSE as  28540.043
restart1
168.4188
168.51398
168.5816
168.73306
168.73428
168.82648
168.78659
168.92117
168.9069
168.92525
this restart has best RMSE as  28540.043
restart2
168.4701
168.53975
168.57394
168.69778
168.74341
168.74954
168.85301
168.87416
168.89462
168.90038
this restart has best RMSE as  28540.043
restart3
168.47813
168.50131
168.62418
168.70708
168.70183
168.78761
168.91055
168.92271
168.93547
168.99207
this restart has best RMSE as  28558.32
restart4
168.4417
168.50433
168.61435
168.70042
168.76445
168.81645
168.86966
168.8257
168.95126
168.94081
this restart has best RMSE as  28558.32
restart0
168.42097
168.61987
168.64638
168.72653
168.82
168.88948
168.99492
169.02554
169.14998
169.10185
this restart has best RMSE as  28611.713
restart1
168.43231
168.57158
168.68784
168.7931
168.88063
168.9311
169.0391
169.02919
169.0849
169.17339
this restart has best RMSE as  28619.637
restart2
168.4

In [88]:
#save alpha=eps=0.01 result to pickle
import pickle
with open('rerun_1k_MSE/attack_dict_pgd.pkl', 'wb') as f:
    pickle.dump(results_pgd, f)